In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

c:\Users\Lenovo\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow_datasets as tfds

# Якщо ви завантажуєте датасет вручну
ds_train, ds_test = tfds.load(
    "mnist",
    split=["train", "test"],
    data_dir='path_to_downloaded_data',
    as_supervised=True,
    with_info=True
)




In [3]:
(ds_train, ds_test), ds_info = tfds.load(
# ds = tfds.load(
    name="mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
    # data_dir=None  # Якщо у вас є локальна директорія з даними, вкажіть її тут
    data_dir='path_to_downloaded_data',

)
# print(ds_info)


In [4]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label


In [5]:

AUTOTUNE = tf.data.AUTOTUNE 
BATCH_SIZE = 32

# Setup for test Dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)# другий параметр щоб паралельно запустити
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
# ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)


In [6]:
model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        tf.keras.layers.Dense(10),
    ]
)

In [15]:
# зберегти лише найкращу модель з усіх епох або як тут всі щоб потім вибрати
save_callback= keras.callbacks.ModelCheckpoint( 
    'checkpoints/model_epoch_{epoch:02d}.weights.h5',  # Ім'я файлу з номером епохи
    save_weights_only=True,
    monitor='accuracy',
    save_best_only=False
    
)

def scheduler(epoch, lr):
    if epoch <2:
        return lr
    else:
        return lr*0.099
    
lr_scheduler = keras.callbacks.LearningRateScheduler(
    scheduler,
    verbose=1
)

class CustomCallback(keras.callbacks.Callback):
    # def on_epoch_end(self, epoch, logs=None):
    #     # print (logs.keys())
    #     if logs.get("accuracy") > 0.90:
    #         print("Accuracy over 90%, quitinig training")
    #         self.model.stop_training = True    
    def on_batch_end(self, epoch, logs=None):
        # print (logs.keys())
        if logs.get("accuracy") > 0.90:
            print("Accuracy over 90%, quitinig training")
            self.model.stop_training = True    
        

In [16]:
model.compile(
    optimizer=keras.optimizers.Adam(0.01),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [17]:
model.fit(
    ds_train,
    epochs=10,
    verbose=1,
    callbacks=[save_callback, lr_scheduler,CustomCallback()]
)


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/10


Accuracy over 90%, quitinig training
1875/1875 [==============================] - 0s 13us/step - loss: 8.1203e-04 - accuracy: 1.0000 - lr: 0.0100
